In [ ]:
import xarray as xr

import qnt.ta as qnta
import qnt.backtester as qnbt
import qnt.data as qndata



def load_data(period):
    return qndata.stocks.load_ndx_data(tail=period)


def strategy(data):
    close     = data.sel(field="close")
    is_liquid = data.sel(field="is_liquid")
    sma_slow  = qnta.sma(close, 150).isel(time=-1)
    sma_fast  = qnta.sma(close, 20).isel(time=-1)
    weights   = xr.where(sma_slow < sma_fast, 1, -1)
    weights   = weights * is_liquid
    return weights

weights = qnbt.backtest(
    competition_type = "stocks_nasdaq100",
    load_data        = load_data,
    lookback_period  = 365*4,
    start_date       = "2022-04-18",
    strategy         = strategy,
    analyze          = True,
    build_plots      = True
)

